## Сборка текстов анекдотов

In [1]:
!pip install fake-useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=57b3cda4b4192eb992e54ce37d5d9e64c68be951b414ac17d44a62d0593c9ee3
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [2]:
import sqlite3
from bs4 import BeautifulSoup
import re
import requests
from pprint import pprint

In [3]:
session = requests.session()

In [4]:
from fake_useragent import UserAgent

ua = UserAgent(verify_ssl=False)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Скачиваем анекдоты.

In [5]:
anek_types = ['vova', 'adults', 'computers', 'blondes', 'piter', 'black', 'kinder', 'gabrovska', 'medical', 'scool']

In [6]:
def load_aneks(anek_types):
    all_aneks = []
    for anek_type in anek_types:
        for i in range(15):
            url = f'https://allanecdots.ru/{anek_type}/{i}/'
            req = session.get(url,  headers={'User-Agent': ua.random})
            page = req.text
            soup = BeautifulSoup(page, 'html.parser')   
            textaneka = soup.find('div', attrs={'class':"container"})
            textaneka2 = textaneka.find_all('p', attrs={'class':"anekdot_body"})
            for smth in textaneka2:
                all_aneks.append({'title': f'https://allanecdots.ru/{anek_type}/{i}/', 'text': re.sub(r'allanecdots.ru', r'', smth.text)})
    return all_aneks

In [7]:
aneks = load_aneks(anek_types)

Удаляем лишние символы.

In [9]:
for i in range(len(aneks)):
    aneks[i]['text'] = aneks[i]['text'].replace('\r', ' ').replace('\n', ' ')

Смотрим.

In [10]:
len(aneks)

1103

In [11]:
aneks[0:10]

[{'title': 'https://allanecdots.ru/vova/1/',
  'text': 'Мама спрашивает Вовочку:  - Кто съел все конфеты?   - Домовой.  Голос из-за печки:  - Не гони!'},
 {'title': 'https://allanecdots.ru/vova/1/',
  'text': 'Учительница:  - Вовочка, назови слово на букву «п».   - Вчера.  - Садись, двойка.  - Почему? Вчера же был понедельник!'},
 {'title': 'https://allanecdots.ru/vova/1/',
  'text': 'На уроке русского языка диктант. Мариванна диктует предложение:  - В углу скребет мышь.   Вовочка (тянет руку):  - Мариванна, ну Мариванна...  Мариванна:  - Что тебе, Вовочка?  Вовочка:  - Мариванна, а кто такой Вуглускр?'},
 {'title': 'https://allanecdots.ru/vova/1/',
  'text': 'Учитeльницa:  - Boвoчкa, нaзoви будущee вpeмя глaгoлa "укpacть".   - Пocaдить.'},
 {'title': 'https://allanecdots.ru/vova/2/',
  'text': '– Мариванна, если всё сущее создал Бог, то кто создал самого Бога?  – Вовочка, ещё один намёк на Путина – и ты вылетишь из класса! '},
 {'title': 'https://allanecdots.ru/vova/2/',
  'text': 'Во

## Делаем корпус

In [12]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 8.2 MB 9.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=86749c86f698dda5893ef085acec6d6f569f0caf29fc178ae19d8975aedacc02
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


Тексты для обработки - это список словарей, в каждом из которых хранится заголовок (сурс) и собственно текст.

In [13]:
aneks[0]

{'title': 'https://allanecdots.ru/vova/1/',
 'text': 'Мама спрашивает Вовочку:  - Кто съел все конфеты?   - Домовой.  Голос из-за печки:  - Не гони!'}

Делаем корпус из этой штуки.

In [14]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [16]:
from tqdm import tqdm

In [17]:
def make_a_corpus(texts):
    corpus = []
    for text in tqdm(texts):
        for sent in sent_tokenize(text['text']):
            new_sent = {}
            new_sent['sentence'] = sent
            tokens = [token.lower() for token in word_tokenize(sent)]
            tokens = list(filter(lambda token: 'PNCT' not in morph.parse(token)[0].tag, tokens)) # поиск не будет учитывать пунктуацию
            new_sent['token'] = tokens
            new_sent['lemma'] = [morph.parse(token)[0].normal_form for token in tokens]
            new_sent['pos'] = [morph.parse(token)[0].tag.POS if isinstance(morph.parse(token)[0].tag.POS, str) else str(morph.parse(token)[0].tag) for token in tokens]
            new_sent['source'] = text['title']
            corpus.append(new_sent)
    return corpus

In [18]:
corpus = make_a_corpus(aneks)

100%|██████████| 1103/1103 [00:36<00:00, 29.83it/s]


Корпус - список словарей. В каждом словаре хранится текст предложения, списки токенов, лемм и частеречных тегов и сурс.

In [19]:
corpus[0]

{'sentence': 'Мама спрашивает Вовочку:  - Кто съел все конфеты?',
 'token': ['мама', 'спрашивает', 'вовочку', 'кто', 'съел', 'все', 'конфеты'],
 'lemma': ['мама', 'спрашивать', 'вовочка', 'кто', 'съесть', 'всё', 'конфета'],
 'pos': ['NOUN', 'VERB', 'NOUN', 'NPRO', 'VERB', 'PRCL', 'NOUN'],
 'source': 'https://allanecdots.ru/vova/1/'}

Корпус сохраняем в джейсон.

In [20]:
import json

In [21]:
with open('corpus.json', 'w', encoding='utf-8') as f_corp:
    json.dump(corpus, f_corp)

## Реализуем поиск

Функция считывания запроса.

In [ ]:
def parse_a_query(query):
    parsed_query = []
    for big_part in query.split():
        new_item = {}
        for part in big_part.split('+'):
            if part[0] == '"' and part[-1] == '"':
                new_item['token'] = part[1:-1]
            elif part.isupper():
                new_item['pos'] = part
            else:
                new_item['lemma'] = part
        parsed_query.append(new_item)
    return parsed_query

Работает это так.

In [ ]:
query0 = 'знать+VERB "осенью" дуб'
parse_a_query(query0)

[{'lemma': 'знать', 'pos': 'VERB'}, {'token': 'осенью'}, {'lemma': 'дуб'}]

Собственно функция поиска берет запрос, парсит, дальше идет по предложениям корпуса и сопоставляет информацию в запросе и то что в предложении собственно есть. Совпадения и соответствующие предложения печатаются.

In [ ]:
def search(query, corpus):

    replys = []
    num_matches = 0
    query = parse_a_query(query)

    if query == []:
        replys.append('Ошибка: пустой запрос.')
        return replys

    for sent in corpus:
        if len(sent['token']) < len(query):
            continue
        for i in range(len(sent['token']) - len(query) + 1):
            match = [] # чтобы печатать совпадение, опционально
            for j in range(len(query)):
                match.append(sent['token'][i+j]) # будут печататься токены, соответствующие запросу
                for key in query[j]:
                    if query[j][key] != sent[key][i+j]:
                        break
                else:
                    continue
                break
            else:
                replys.append('Совпадение: ' + ' '.join(match) +'. Предложение: '+'\n' + sent['sentence'] + '\n' + 'Источник: ' + sent['source'] + '\n\n')
                num_matches += 1
                # break # каждое предложение из корпуса будет печататься 1 раз максимум (можно включить и убрать печать совпадений)
                
    if num_matches == 0:
        replys.append('По этому запросу ничего не найдено.')
    else:
        replys.append('Итого найдено ' + str(num_matches) + ' совпадений.')
    return replys

## Тестим поиск

Открываем корпус.

In [ ]:
with open('corpus.json', 'r', encoding='utf-8') as f_corp:
    corpus = json.load(f_corp)

Смотрим.

In [ ]:
corpus[0]

{'sentence': 'Мама спрашивает Вовочку:  - Кто съел все конфеты?',
 'token': ['мама', 'спрашивает', 'вовочку', 'кто', 'съел', 'все', 'конфеты'],
 'lemma': ['мама', 'спрашивать', 'вовочка', 'кто', 'съесть', 'всё', 'конфета'],
 'pos': ['NOUN', 'VERB', 'NOUN', 'NPRO', 'VERB', 'PRCL', 'NOUN'],
 'source': 'https://allanecdots.ru/vova/1/'}

Ищем.

In [ ]:
def print_search(query, corpus):
    for reply in search(query, corpus):
        print(reply)

In [ ]:
print_search('мама', corpus)

Совпадение: мама
Мама спрашивает Вовочку:  - Кто съел все конфеты?
Источник: https://allanecdots.ru/vova/1/


Совпадение: мама
Мама говорит Вовочке:  - Сынок, что нужно сказать тете Клаве?
Источник: https://allanecdots.ru/vova/2/


Совпадение: мама
– Мариванна, мама мои трусы постирала, а других у меня нет!
Источник: https://allanecdots.ru/vova/3/


Совпадение: мама
Мама Вовочке:  - Садись и расскажи нам какую-нибудь забавную историю.
Источник: https://allanecdots.ru/vova/3/


Совпадение: мама
Вовочка - матери:  - Мама, я братика хочу.
Источник: https://allanecdots.ru/vova/3/


Совпадение: мама
Мама спрашивает Вовочку:  - Кто съел все конфеты?
Источник: https://allanecdots.ru/vova/3/


Совпадение: мама
Мама читает Вовочке сказку.
Источник: https://allanecdots.ru/vova/3/


Совпадение: маме
Вовочка - маме:  - Почему папа лысый?
Источник: https://allanecdots.ru/vova/3/


Совпадение: мамы
- Смотри, у меня нога сорок четвертого размера, а у мамы тридцать шестого.
Источник: https://allanecdo

In [ ]:
print_search('"маму"', corpus)

Совпадение: маму
- Да, у тебя сзади написано "Ешь не чавкай", а спереди "Расти большой и слушай маму!"
Источник: https://allanecdots.ru/piter/7/


Совпадение: маму
Маму разбудишь!
Источник: https://allanecdots.ru/black/7/


Совпадение: маму
Мальчик, увидев их, притих, очень внимательно рассматривал, а потом спросил маму:  - А где мой такой?
Источник: https://allanecdots.ru/kinder/9/


Совпадение: маму
У маленького мальчика спрашивают: - Ты кого больше слушаешь, маму или папу?
Источник: https://allanecdots.ru/kinder/14/


Совпадение: маму
- Маму!
Источник: https://allanecdots.ru/kinder/14/


Итого найдено 5 совпадений.


In [ ]:
print_search('ADJF', corpus)

Выходные данные были обрезаны до нескольких последних строк (5000).
- Я хочу, чтобы через год после моей смерти ты вышла замуж за Семена.
Источник: https://allanecdots.ru/black/3/


Совпадение: последнем
На последнем дыхании Петр сказал:  - Еще как ненавижу...
Источник: https://allanecdots.ru/black/3/


Совпадение: новых
Это вы тут в заявке требуете десять новых холодильников?
Источник: https://allanecdots.ru/black/3/


Совпадение: хозяйственном
Например, веревки в хозяйственном отделе, мыло в косметическом, табуретки вообще на другом этаже, в мебельном.
Источник: https://allanecdots.ru/black/4/


Совпадение: косметическом
Например, веревки в хозяйственном отделе, мыло в косметическом, табуретки вообще на другом этаже, в мебельном.
Источник: https://allanecdots.ru/black/4/


Совпадение: мебельном
Например, веревки в хозяйственном отделе, мыло в косметическом, табуретки вообще на другом этаже, в мебельном.
Источник: https://allanecdots.ru/black/4/


Совпадение: лучшего
Лучшего работника

In [ ]:
print_search('странный+ADJF', corpus)

Совпадение: странные
Жалобы - странные звуки при работе.
Источник: https://allanecdots.ru/computers/4/


Совпадение: странным
Вечером, за ужином, врач похохатывая поделился с женой этим странным визитом.
Источник: https://allanecdots.ru/blondes/14/


Совпадение: странное
– А почему такое странное имя?!
Источник: https://allanecdots.ru/kinder/9/


Итого найдено 3 совпадений.


In [ ]:
print_search('PREP NOUN VERB', corpus)

Совпадение: в углу скребет
Мариванна диктует предложение:  - В углу скребет мышь.
Источник: https://allanecdots.ru/vova/1/


Совпадение: от пьянства кодировался
Вовочка: - Он три раза от пьянства кодировался и три раза код ломал.
Источник: https://allanecdots.ru/vova/2/


Совпадение: до революции говорили
Вовочка:  - До революции говорили: "Пшел вон!"
Источник: https://allanecdots.ru/vova/2/


Совпадение: на вовочку договорила
И поглядев с опаской на Вовочку договорила:  - На букву Х, в котором нет буквы У.  Вовочка.
Источник: https://allanecdots.ru/vova/2/


Совпадение: по члену бегает
Голова трещит, ничего не соображает:   – Слышь, Хоттабыч, а че это у меня по члену бегает и свистит?
Источник: https://allanecdots.ru/vova/2/


Совпадение: на зиму улетают
На уроке природоведения учительница спрашивает:  - Кто знает, почему аисты на зиму улетают в Африку?
Источник: https://allanecdots.ru/vova/3/


Совпадение: на выключатель нажмешь
На выключатель нажмешь - и тут же свет загорается.
Исто

In [ ]:
print_search('странный+ADJF NOUN', corpus)

Совпадение: странные звуки
Жалобы - странные звуки при работе.
Источник: https://allanecdots.ru/computers/4/


Совпадение: странным визитом
Вечером, за ужином, врач похохатывая поделился с женой этим странным визитом.
Источник: https://allanecdots.ru/blondes/14/


Совпадение: странное имя
– А почему такое странное имя?!
Источник: https://allanecdots.ru/kinder/9/


Итого найдено 3 совпадений.


In [ ]:
print_search('"такое" странный+ADJF NOUN', corpus)

Совпадение: такое странное имя
– А почему такое странное имя?!
Источник: https://allanecdots.ru/kinder/9/


Итого найдено 1 совпадений.


In [ ]:
print_search('mecomoJKDOKWOkd/18767Y* 00i0wfpm', corpus)

По этому запросу ничего не найдено.


In [ ]:
print_search('', corpus)

Ошибка: пустой запрос.
